In [10]:
import os
import timm
import cv2
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import _LRScheduler
from torchvision.transforms.functional import to_pil_image

In [11]:
# def seed_everything(random_seed: int):
#     torch.manual_seed(random_seed)
#     torch.cuda.manual_seed(random_seed)
#     torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
#     torch.backends.cudnn.deterministic = True
#     torch.backends.cudnn.benchmark = False
#     np.random.seed(random_seed)
#     random.seed(random_seed)
#     os.environ['PYTHONHASHSEED'] = str(random_seed)
#     print('seed setting complete')

In [12]:
##### HYPER PARAMETER

SEED = 42
PATCH_SIZE = (512, 512) # width, height
OVERLAP_RATIO = 0.1 #0.5
TISSUE_AREA_RATIO = 0.5
DROP_RATE = 0.5
EPOCH = 10
LEARNING_RATE = 3e-5
WEIGHT_DECAY = 1e-5

TRAIN_BATCH_SIZE = 64 #64
EVAL_BATCH_SIZE = 64 #256
TOPK = 5 # MIL top K

GPU_IDX = 'cuda'
DATE = '240102'

EXP_NAME = f'MAIC_{DATE}_{LEARNING_RATE}'
seed_everything(SEED)

seed setting complete


In [31]:
model = timm.create_model('resnet18', pretrained=True, num_classes=2, drop_rate=DROP_RATE)

In [ ]:
for name, param in model.named_parameters():
    print(f'Layer: {name}, Shape: {param.shape}')

In [34]:
res18_module = list(model.children())[:-1]
res18_model = nn.Sequential(*res18_module)

In [54]:
input_image = torch.randn(3, 3, 224, 224)
with torch.no_grad():
    res18_model.eval()
    output_feature = res18_model(input_image)

In [55]:
output_feature.size()

torch.Size([3, 512])

In [47]:
list(res18_model.modules())[-1]

Flatten(start_dim=1, end_dim=-1)

In [52]:
def load_dataset_from_pickle(file_name):
    """
    .pkl 파일로부터 데이터셋을 로드합니다.
    
    :param file_name: 로드할 .pkl 파일의 이름
    :return: 파일에서 로드된 데이터셋
    """
    with open(file_name, 'rb') as file:
        dataset = pickle.load(file)
    return dataset

In [53]:
%%time
import pickle

train_data = load_dataset_from_pickle('../image_pickle/dataset_train_001.pkl')

AttributeError: Can't get attribute 'MILDataset' on <module '__main__'>

In [ ]:
import pickle

train_data = load_dataset_from_pickle('../pickle/train_dataset.pkl')
train_train_dataloader = DataLoader(train_data, batch_size=TRAIN_BATCH_SIZE, shuffle=False, num_workers=0)


model = res18_model
with torch.no_grad() :
    model.eval()
    for batch_idx, (data, label) in enumerate(tqdm(train_train_dataloader)):
        data = data.to(GPU_IDX)
        feature_embedding = model(data)
        label = label.numpy()